# **Task-3: Correlation Power Analysis on Simulated Power Traces**

In the last assignment, you learnt how to simulate power traces. In this assignment, you shall be provided with simulated trace files. You have to perform the Correlation Power Analysis (CPA) attack on these traces. The same attack was performed in the "IoT Goes Nuclear" paper and in several other practial scenarios. 

In the theory sessions, we have discussed the CPA attack algorithm. In this assignment you will be implementing the same.

# **Download the Prerequisites**

In [ ]:
!wget "https://tinyurl.com/y2lttorc"
!unzip y2lttorc
!rm y2lttorc

!wget "https://tinyurl.com/y596slbw"
!unzip y596slbw
!rm y596slbw

!wget "https://tinyurl.com/yywympeh"
!unzip yywympeh
!rm yywympeh

!wget "https://tinyurl.com/yy6ucxdz"
!unzip yy6ucxdz
!rm yy6ucxdz

!wget "https://tinyurl.com/y3aerc86"
!unzip y3aerc86
!rm y3aerc86

# **Instructions**

# **Part 1: Implement CPA Algorithm**
In the following code cell: 

1. Replace "Your_code" at **line no. 60 and 61** with correct integer values.

2. Replace "Your_code" at **line no. 69, 71, 73, and 74** with proper expressions. **line no. 71** is for HW leakage model, and **line no. 73, 74** are for HD leakage model.
(**Note**: At some point you may need to perform an inverse S-Box operation. The function for that is:


> `<output_byte>=cipher.cipher.sbox_inv[<input_byte>]`


3. Run the code on noise-free traces with HW leakage model (default) and report the correct key value printed on the console. Also, carefully observe the plot generated.

# **The CPA Code**

In [ ]:
import math
import numpy as np
import scipy
import random
import csv
import matplotlib.pyplot as plt
from aes_cpa import *
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# Compute the HW of an integer
def HW(val):
	h = 0
	while( not(val == 0) ):
		v = val%2
		h = h + v
		val = int(val/2)
	return h
#-----------------------------------------------------------------------------------------------------------------------------------------------------
filename = 'trace_file_hw.csv'

numTraces = 400			  # Number of traces used for attack
leakage_model = 0			# leakage_model = 0 --> HW; leakage_model = 1 --> HD
target_byte = 0
target_byte_acc = target_byte
wlen = 12

cipher = AES()
cipher.KeyExpansion()
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# Initialize necessary data structures
hypo_vector = np.zeros(numTraces,dtype='float')
pt_vector = np.zeros(numTraces,dtype='float')
trace = np.zeros((numTraces,wlen),dtype='float')
ct_list = []

#-----------------------------------------------------------------------------------------------------------------------------------------------------
# Read the trace file
with open(filename, mode='r') as trace_file:
	i = 0
	trace_reader = csv.reader(trace_file, delimiter=',')
	for row in trace_reader:
		ct_list.append(hexstringtointarray(row[0]))	
		trace[i, :] = row[1:13]
		i = i + 1
		if i == numTraces: break
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# Perfrom the CPA
numrow = 256       	# Number of rows in the correlation matrix (Fill in here)
numcol = 12	   				# Number of columns in the correlation matrix (Fill in here)

corr_matrix=np.zeros((numrow,numcol),dtype='float')	# Initialize the correlation matrix
for c in range(numcol):								
	for r in range(0,numrow,1):							 
		for j in range(numTraces):							
			ct = ct_list[j]
			ct_temp = ct[target_byte_acc] & 0xff                										
			btemp = cipher.sbox_inv[ct_temp^r]		# Fill in here #
			if (leakage_model == 0):  				# HW
				hypo_vector[j]=	HW(btemp) # Fill in here	# 
			else:	                 				# HD
				btemp_hd=btemp^ct_temp		# Fill in here	#                             
				hypo_vector[j]=	HW(btemp_hd)	# Fill in here	# 				                           
		pt_vector=trace[:,c]                                

		if (len(set(pt_vector)) > 1):
			val=np.corrcoef(hypo_vector,pt_vector)
			corr_matrix[r][c]=(abs)(val[0][1])    			
		else:
			val = 0	
			corr_matrix[r][c]= val		          
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# Find the row containing the max correaltion value. The key with max. correlation value is the correct key        
maxval = 0  
for i in range(numrow):
    row=corr_matrix[i,:]
    if(maxval < max(row)):
        maxval = max(row)
        correct_key = i
        correct_row = row
               
print ("correct_key_byte = " + str(hex(correct_key)) )        	
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# Print the correlation values for all the keys and all the points of interest
fig, ax1=plt.subplots()
for i in range(256):
    if(i==correct_key):
        plt.plot(range(wlen),correct_row,'r',linewidth=1.0,label="correct key byte")
    else:
        plt.plot(range(wlen),corr_matrix[i,:],'k',linewidth=0.2,)
        
ax1.legend()
plt.locator_params(axis='y', nbins=5)
plt.title('CPA Plot')
plt.xlabel('Sample Points')
plt.ylabel('Correlation Value')
plt.show()

# **Part 2: Perform CPA on Noise-free HD Traces**
In the following code cell:

1. Copy the previous code cell below this text cell. (Right click on the code cell and select "copy cell". Then press "Ctrl+V". Finally, if required, use the down arrow at the upper right corner of the copied cell to move it down.)  

2. Change line no. 21 to:

> `filename = 'trace_file_hd.csv'`

3. Change line no. 24 to:

> `leakage_model = 1`


> This runs the code for HD leakage model.


4. Run the code.

In [ ]:
import math
import numpy as np
import scipy
import random
import csv
import matplotlib.pyplot as plt
from aes_cpa import *
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# Compute the HW of an integer
def HW(val):
	h = 0
	while( not(val == 0) ):
		v = val%2
		h = h + v
		val = int(val/2)
	return h
#-----------------------------------------------------------------------------------------------------------------------------------------------------
filename = 'trace_file_hd.csv'

numTraces = 400			  # Number of traces used for attack
leakage_model = 1			# leakage_model = 0 --> HW; leakage_model = 1 --> HD
target_byte = 0
target_byte_acc = target_byte
wlen = 12

cipher = AES()
cipher.KeyExpansion()
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# Initialize necessary data structures
hypo_vector = np.zeros(numTraces,dtype='float')
pt_vector = np.zeros(numTraces,dtype='float')
trace = np.zeros((numTraces,wlen),dtype='float')
ct_list = []
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# Read the trace file
with open(filename, mode='r') as trace_file:
	i = 0
	trace_reader = csv.reader(trace_file, delimiter=',')
	for row in trace_reader:
		ct_list.append(hexstringtointarray(row[0]))	
		trace[i, :] = row[1:13]
		i = i + 1
		if i == numTraces: break
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# Perfrom the CPA
numrow = 256       	# Number of rows in the correlation matrix (Fill in here)
numcol = 12	   				# Number of columns in the correlation matrix (Fill in here)

corr_matrix=np.zeros((numrow,numcol),dtype='float')	# Initialize the correlation matrix
for c in range(numcol):								
	for r in range(0,numrow,1):							 
		for j in range(numTraces):							
			ct = ct_list[j]
			ct_temp = ct[target_byte_acc] & 0xff                										
			btemp = cipher.sbox_inv[ct_temp^r]		# Fill in here #
			if (leakage_model == 0):  				# HW
				hypo_vector[j]=	HW(btemp) # Fill in here	# 
			else:	                 				# HD
				btemp_hd=btemp^ct_temp		# Fill in here	#                             
				hypo_vector[j]=	HW(btemp_hd)	# Fill in here	# 				                           
		pt_vector=trace[:,c]                                

		if (len(set(pt_vector)) > 1):
			val=np.corrcoef(hypo_vector,pt_vector)
			corr_matrix[r][c]=(abs)(val[0][1])    			
		else:
			val = 0	
			corr_matrix[r][c]= val		          
#-----------------------------------------------------------------------------------------------------------------------------------------------------       
maxval = 0  
for i in range(numrow):
    row=corr_matrix[i,:]
    if(maxval < max(row)):
        maxval = max(row)
        correct_key = i
        correct_row = row
               
print ("correct_key_byte = " + str(hex(correct_key)) )        	
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# Print the correlation values for all the keys and all the points of interest
fig, ax1=plt.subplots()
for i in range(256):
    if(i==correct_key):
        plt.plot(range(wlen),correct_row,'r',linewidth=1.0,label="correct key byte")
    else:
        plt.plot(range(wlen),corr_matrix[i,:],'k',linewidth=0.2,)
        
ax1.legend()
plt.locator_params(axis='y', nbins=5)
plt.title('CPA Plot')
plt.xlabel('Sample Points')
plt.ylabel('Correlation Value')
plt.show()

# **Part 3: Perform CPA on Noisy Traces**
In the following code cell:

1. Copy the previous code cell below this text cell. (Right click on the code cell and select "copy cell". Then press "Ctrl+V". Finally, if required, use the down arrow at the upper right corner of the copied cell to move it down.)  

2. Change line no. 21 to:

> `filename = 'trace_file_hw_noisy.csv'`

3. Change line no. 24 to:

> `leakage_model = 0`


4. Run the code. You may not recover the correct key byte value (the correct is the one you recovered with noise-free traces). 

5. Increase the number of traces at line no. 23:
(`numTraces = 400`) to by 100 (i.e. make it 500).

6. Observe the impact. 

7. Continue increasing the number of traces until you recover the correct key byte.

8. One you are done with the HW traces, create another code cell and repeat the experiment for noisy HD traces. 



In [ ]:
import math
import numpy as np
import scipy
import random
import csv
import matplotlib.pyplot as plt
from aes_cpa import *
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# Compute the HW of an integer
def HW(val):
	h = 0
	while( not(val == 0) ):
		v = val%2
		h = h + v
		val = int(val/2)
	return h
#-----------------------------------------------------------------------------------------------------------------------------------------------------
filename = 'trace_file_hw_noisy.csv'

numTraces = 900			  # Number of traces used for attack
leakage_model = 0			# leakage_model = 0 --> HW; leakage_model = 1 --> HD
target_byte = 0
target_byte_acc = target_byte
wlen = 12

cipher = AES()
cipher.KeyExpansion()
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# Initialize necessary data structures
hypo_vector = np.zeros(numTraces,dtype='float')
pt_vector = np.zeros(numTraces,dtype='float')
trace = np.zeros((numTraces,wlen),dtype='float')
ct_list = []
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# Read the trace file
with open(filename, mode='r') as trace_file:
	i = 0
	trace_reader = csv.reader(trace_file, delimiter=',')
	for row in trace_reader:
		ct_list.append(hexstringtointarray(row[0]))	
		trace[i, :] = row[1:13]
		i = i + 1
		if i == numTraces: break
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# Perfrom the CPA
numrow = 256       	# Number of rows in the correlation matrix (Fill in here)
numcol = 12	   				# Number of columns in the correlation matrix (Fill in here)

corr_matrix=np.zeros((numrow,numcol),dtype='float')	# Initialize the correlation matrix
for c in range(numcol):								
	for r in range(0,numrow,1):							 
		for j in range(numTraces):							
			ct = ct_list[j]
			ct_temp = ct[target_byte_acc] & 0xff                										
			btemp = cipher.sbox_inv[ct_temp^r]		# Fill in here #
			if (leakage_model == 0):  				# HW
				hypo_vector[j]=	HW(btemp) # Fill in here	# 
			else:	                 				# HD
				btemp_hd=btemp^ct_temp		# Fill in here	#                             
				hypo_vector[j]=	HW(btemp_hd)	# Fill in here	# 				                           
		pt_vector=trace[:,c]                                

		if (len(set(pt_vector)) > 1):
			val=np.corrcoef(hypo_vector,pt_vector)
			corr_matrix[r][c]=(abs)(val[0][1])    			
		else:
			val = 0	
			corr_matrix[r][c]= val		          
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# Find the row containing the max correaltion value. The key with max. correlation value is the correct key        
maxval = 0  
for i in range(numrow):
    row=corr_matrix[i,:]
    if(maxval < max(row)):
        maxval = max(row)
        correct_key = i
        correct_row = row
               
print ("correct_key_byte = " + str(hex(correct_key)) )        	
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# Print the correlation values for all the keys and all the points of interest
fig, ax1=plt.subplots()
for i in range(256):
    if(i==correct_key):
        plt.plot(range(wlen),correct_row,'r',linewidth=1.0,label="correct key byte")
    else:
        plt.plot(range(wlen),corr_matrix[i,:],'k',linewidth=0.2,)
        
ax1.legend()
plt.locator_params(axis='y', nbins=5)
plt.title('CPA Plot')
plt.xlabel('Sample Points')
plt.ylabel('Correlation Value')
plt.show()
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# correct_key_byte = 0xd0
# at numTraces=400 ----> correct_key_byte = 0xb3
# at numTraces=500 ----> correct_key_byte = 0xef
# at numTraces=600 ----> correct_key_byte = 0xef
# at numTraces=700 ----> correct_key_byte = 0xb0
# at numTraces=800 ----> correct_key_byte = 0x84
# at numTraces=900 ----> correct_key_byte = 0xd0

In [ ]:
import math
import numpy as np
import scipy
import random
import csv
import matplotlib.pyplot as plt
from aes_cpa import *
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# Compute the HW of an integer
def HW(val):
	h = 0
	while( not(val == 0) ):
		v = val%2
		h = h + v
		val = int(val/2)
	return h
#-----------------------------------------------------------------------------------------------------------------------------------------------------
filename = 'trace_file_hd_noisy.csv'

numTraces = 1200			  # Number of traces used for attack
leakage_model = 1			# leakage_model = 0 --> HW; leakage_model = 1 --> HD
target_byte = 0
target_byte_acc = target_byte
wlen = 12

cipher = AES()
cipher.KeyExpansion()
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# Initialize necessary data structures
hypo_vector = np.zeros(numTraces,dtype='float')
pt_vector = np.zeros(numTraces,dtype='float')
trace = np.zeros((numTraces,wlen),dtype='float')
ct_list = []
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# Read the trace file
with open(filename, mode='r') as trace_file:
	i = 0
	trace_reader = csv.reader(trace_file, delimiter=',')
	for row in trace_reader:
		ct_list.append(hexstringtointarray(row[0]))	
		trace[i, :] = row[1:13]
		i = i + 1
		if i == numTraces: break
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# Perfrom the CPA
numrow = 256       	# Number of rows in the correlation matrix (Fill in here)
numcol = 12	   				# Number of columns in the correlation matrix (Fill in here)

corr_matrix=np.zeros((numrow,numcol),dtype='float')	# Initialize the correlation matrix
for c in range(numcol):								
	for r in range(0,numrow,1):							 
		for j in range(numTraces):							
			ct = ct_list[j]
			ct_temp = ct[target_byte_acc] & 0xff                										
			btemp = cipher.sbox_inv[ct_temp^r]		# Fill in here #
			if (leakage_model == 0):  				# HW
				hypo_vector[j]=	HW(btemp) # Fill in here	# 
			else:	                 				# HD
				btemp_hd=btemp^ct_temp		# Fill in here	#                             
				hypo_vector[j]=	HW(btemp_hd)	# Fill in here	# 				                           
		pt_vector=trace[:,c]                                

		if (len(set(pt_vector)) > 1):
			val=np.corrcoef(hypo_vector,pt_vector)
			corr_matrix[r][c]=(abs)(val[0][1])    			
		else:
			val = 0	
			corr_matrix[r][c]= val		          
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# Find the row containing the max correaltion value. The key with max. correlation value is the correct key        
maxval = 0  
for i in range(numrow):
    row=corr_matrix[i,:]
    if(maxval < max(row)):
        maxval = max(row)
        correct_key = i
        correct_row = row
               
print ("correct_key_byte = " + str(hex(correct_key)) )        	
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# Print the correlation values for all the keys and all the points of interest
fig, ax1=plt.subplots()
for i in range(256):
    if(i==correct_key):
        plt.plot(range(wlen),correct_row,'r',linewidth=1.0,label="correct key byte")
    else:
        plt.plot(range(wlen),corr_matrix[i,:],'k',linewidth=0.2,)
        
ax1.legend()
plt.locator_params(axis='y', nbins=5)
plt.title('CPA Plot')
plt.xlabel('Sample Points')
plt.ylabel('Correlation Value')
plt.show()
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# correct_key_byte = 0xd0
# at numTraces=400 ----> correct_key_byte = 0x1e
# at numTraces=500 ----> correct_key_byte = 0x1e
# at numTraces=600 ----> correct_key_byte = 0x94
# at numTraces=700 ----> correct_key_byte = 0x94
# at numTraces=800 ----> correct_key_byte = 0x94
# at numTraces=900 ----> correct_key_byte = 0x94
# at numTraces=1000----> correct_key_byte = 0x94
# at numTraces=1100----> correct_key_byte = 0x94
# at numTraces=1200----> correct_key_byte = 0xd0
# Final Answer --------> 1200